<font face='georgia'>
    
   <h4><strong>What does tf-idf mean?</strong></h4>

   <p>    
Tf-idf stands for <em>term frequency-inverse document frequency</em>, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.
</p>
    
   <p>
One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.
</p>
    
   <p>
Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.
</p>
    
</font>

<font face='georgia'>
    <h4><strong>How to Compute:</strong></h4>

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

 <ul>
    <li>
<strong>TF:</strong> Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization: <br>

$TF(t) = \frac{\text{Number of times term t appears in a document}}{\text{Total number of terms in the document}}.$
</li>
<li>
<strong>IDF:</strong> Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weigh down the frequent terms while scale up the rare ones, by computing the following: <br>

$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}}.$
for numerical stabiltiy we will be changing this formula little bit
$IDF(t) = \log_{e}\frac{\text{Total  number of documents}} {\text{Number of documents with term t in it}+1}.$
</li>
</ul>

<br>
<h4><strong>Example</strong></h4>
<p>

Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.
</p>
</font>

## Task-1

<font face='georgia'>
    <h4><strong>1. Build a TFIDF Vectorizer & compare its results with Sklearn:</strong></h4>

<ul>
    <li> As a part of this task you will be implementing TFIDF vectorizer on a collection of text documents.</li>
    <br>
    <li> You should compare the results of your own implementation of TFIDF vectorizer with that of sklearns implemenation TFIDF vectorizer.</li>
    <br>
    <li> Sklearn does few more tweaks in the implementation of its version of TFIDF vectorizer, so to replicate the exact results you would need to add following things to your custom implementation of tfidf vectorizer:
       <ol>
        <li> Sklearn has its vocabulary generated from idf sroted in alphabetical order</li>
        <li> Sklearn formula of idf is different from the standard textbook formula. Here the constant <strong>"1"</strong> is added to the numerator and denominator of the idf as if an extra document was seen containing every term in the collection exactly once, which prevents zero divisions.
            
 $IDF(t) = 1+\log_{e}\frac{1\text{ }+\text{ Total  number of documents in collection}} {1+\text{Number of documents with term t in it}}.$
        </li>
        <li> Sklearn applies L2-normalization on its output matrix.</li>
        <li> The final output of sklearn tfidf vectorizer is a sparse matrix.</li>
    </ol>
    <br>
    <li>Steps to approach this task:
    <ol>
        <li> You would have to write both fit and transform methods for your custom implementation of tfidf vectorizer.</li>
        <li> Print out the alphabetically sorted voacb after you fit your data and check if its the same as that of the feature names from sklearn tfidf vectorizer. </li>
        <li> Print out the idf values from your implementation and check if its the same as that of sklearns tfidf vectorizer idf values. </li>
        <li> Once you get your voacb and idf values to be same as that of sklearns implementation of tfidf vectorizer, proceed to the below steps. </li>
        <li> Make sure the output of your implementation is a sparse matrix. Before generating the final output, you need to normalize your sparse matrix using L2 normalization. You can refer to this link https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html </li>
        <li> After completing the above steps, print the output of your custom implementation and compare it with sklearns implementation of tfidf vectorizer.</li>
        <li> To check the output of a single document in your collection of documents,  you can convert the sparse matrix related only to that document into dense matrix and print it.</li>
        </ol>
    </li>
    <br>
   </ul>

# Task - 1

In [2]:
# Importing essential libraries
import numpy as np
from sklearn.preprocessing import normalize
from scipy.sparse import csr_matrix
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter

In [3]:
corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

# Writing fit() method

In [4]:
def fit(corpus):    
# 1. Collecting unique words from corpus and sorting them

    unique_words = set()
    if isinstance(corpus, (list,)):
        for row in corpus:
            for word in row.split(' '):
                if len(word)<2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))

    return unique_words

In [5]:
unique_words = fit(corpus)
print(unique_words)

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


# Writing transform() method

In [26]:
def transform(corpus, unique_words):
    from tqdm import tqdm
    import numpy as np
    from scipy.sparse import csr_matrix
    from sklearn.preprocessing import normalize
# 1. Creating list of documents inside documents for simplification only
    lst_0 = list()
    j = 0
    for i in corpus:
        lst_0.append([])
        lst_0[j].append(i)
        j += 1

# 2. Separating each word to iterate amongst words nicely
    lst_1 = []
    for i in lst_0:
        for j in i:
            lst_1.append((j.split(' ')))
    
# 3. Computing tf -
    print('idf values for given corpus =')
    i = 0
    Values = []
    for sent in range(len(lst_1)):
        Values.append([])
        for word in range(len(unique_words)):
            tf = lst_1[sent].count(unique_words[word])/len(lst_1[sent])

# 4. computing Idf-

            N = len(lst_1)     # total no. of documents in corpus

            n=0                # no. of doc has term-'t'
            for i in range(len(lst_1)):
                if lst_1[i].count(unique_words[word]) != 0:
                    n += 1

            idf = 1 + (np.log((1+N)/(1+n)))
            
            
# 5. Computing Tf-Idf values

            Tf_Idf = tf*idf
            Values[sent].append(Tf_Idf)
        
# ************ I made changes here to adopt changes you suggested : ************
# ************ Suggestion: Please print the idf values along with the feature names ************
            if sent == 0:
                print(unique_words[word],':',idf)
        
        
# 6. normalizing
    normalized_val = normalize(Values)
    
    
# 7. Creating sparse matrix
    sparse_val = csr_matrix(normalized_val)
    return sparse_val

In [27]:
a = transform(corpus, unique_words)
print('\nType of custom TfIdfVec output matrix =', type(a), '\n')
print('printing some values of tfidf =\n',a[0], '\n')
print('Sparse matrix to Feature matrix =\n',a.toarray())

idf values for given corpus =
and : 1.916290731874155
document : 1.2231435513142097
first : 1.5108256237659907
is : 1.0
one : 1.916290731874155
second : 1.916290731874155
the : 1.0
third : 1.916290731874155
this : 1.0

Type of custom TfIdfVec output matrix = <class 'scipy.sparse.csr.csr_matrix'> 

printing some values of tfidf =
   (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149 

Sparse matrix to Feature matrix =
 [[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


### Implementing sklearn TfidfVectorizer() - For cross verifying outputs

In [12]:
# 1. fit and transform
from sklearn.feature_extraction.text import TfidfVectorizer
vec = TfidfVectorizer().fit(corpus)
X = vec.transform(corpus)

In [13]:
# 2. printing feature names
print(vec.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [14]:
# 3. printing idf values of sklearn vectirizer
print(vec.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [15]:
# 4. sklearn has sparse presentation as output matrix
type(X)

scipy.sparse.csr.csr_matrix

In [16]:
# 5. printing sklearn output for individual document
print(X[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [17]:
# 6. Sparse matrix (sklearn) to Feature matrix 
print(X.toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


In [ ]:
# **************** Task-1 Finishes here ****************

# Task - 2

<font face='georgia'>
    <h4><strong>2. Implement max features functionality:</strong></h4>

<ul>
    <li> As a part of this task you have to modify your fit and transform functions so that your vocab will contain only 50 terms with top idf scores.</li>
    <br>
    <li>This task is similar to your previous task, just that here your vocabulary is limited to only top 50 features names based on their idf values. Basically your output will have exactly 50 columns and the number of rows will depend on the number of documents you have in your corpus.</li>
    <br>
    <li>Here you will be give a pickle file, with file name <strong>cleaned_strings</strong>. You would have to load the corpus from this file and use it as input to your tfidf vectorizer.</li>
    <br>
    <li>Steps to approach this task:
    <ol>
        <li> You would have to write both fit and transform methods for your custom implementation of tfidf vectorizer, just like in the previous task. Additionally, here you have to limit the number of features generated to 50 as described above.</li>
        <li> Now sort your vocab based in descending order of idf values and print out the words in the sorted voacb after you fit your data. Here you should be getting only 50 terms in your vocab. And make sure to print idf values for each term in your vocab. </li>
        <li> Make sure the output of your implementation is a sparse matrix. Before generating the final output, you need to normalize your sparse matrix using L2 normalization. You can refer to this link https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.normalize.html </li>
        <li> Now check the output of a single document in your collection of documents,  you can convert the sparse matrix related only to that document into dense matrix and print it. And this dense matrix should contain 1 row and 50 columns. </li>
        </ol>
    </li>
    <br>
   </ul>

In [19]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open('cleaned_strings', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


# Writing fit() method

In [20]:
def fit(corpus):
    # 0. all Importing essential libraries
    import numpy as np
    from scipy.sparse import csr_matrix
    from tqdm import tqdm
    from collections import Counter
    
# 1. Creating list of documents inside documents for simplification only
    lst_0 = list()
    j = 0
    for i in corpus:
        lst_0.append([])
        lst_0[j].append(i)
        j += 1
    
# 2. Separating each word to iterate amongst words nicely
    lst_1 = []
    for i in lst_0:
        for j in i:
            lst_1.append((j.split(' ')))
    
# 3. Collecting unique words from corpus and sorting them
    unique_words = set()
    if isinstance(corpus, (list,)):
        for row in corpus:
            for word in row.split(' '):
                if len(word)<2:
                    continue
                unique_words.add(word)
        unique_words = sorted(list(unique_words))

# 4. computing idf values for corpus........

    i = 0
    idf_val = []
    N = len(lst_1)             # total no. of documents in corpus
    for sent in range(len(lst_1)):
        for word in range(len(unique_words)):
        
            n=0                # no. of doc has term-'t'
            for i in range(len(lst_1)):
                if lst_1[i].count(unique_words[word]) != 0:
                    n += 1

            idf = 1 + (np.log((1+N)/(1+n)))
            idf_val.append(idf)
        break
    my_idf = np.array(idf_val)
    
# 5. Creating max_feature
    vocab_0 = dict(zip(unique_words, idf_val))

    sorted(vocab_0.values())
    a = Counter(vocab_0)
    vocab = a.most_common(50)

    vocab = dict(vocab)
    
    max_feature = list(vocab.keys())
    return max_feature

In [22]:
max_feature = fit(corpus)
print(max_feature)

['aailiyah', 'abandoned', 'abroad', 'abstruse', 'academy', 'accents', 'accessible', 'acclaimed', 'accolades', 'accurate', 'accurately', 'achille', 'ackerman', 'actions', 'adams', 'add', 'added', 'admins', 'admiration', 'admitted', 'adrift', 'adventure', 'aesthetically', 'affected', 'affleck', 'afternoon', 'aged', 'ages', 'agree', 'agreed', 'aimless', 'aired', 'akasha', 'akin', 'alert', 'alike', 'allison', 'allow', 'allowing', 'alongside', 'amateurish', 'amaze', 'amazed', 'amazingly', 'amusing', 'amust', 'anatomist', 'angel', 'angela', 'angelina']


# Writing transform() method

In [23]:

def transform(corpus, max_feature):
    from tqdm import tqdm
    import numpy as np
    from scipy.sparse import csr_matrix
    from sklearn.preprocessing import normalize
# 1. Creating list of documents inside documents for simplification only
    lst_0 = list()
    j = 0
    for i in corpus:
        lst_0.append([])
        lst_0[j].append(i)
        j += 1

# 2. Separating each word to iterate amongst words nicely
    lst_1 = []
    for i in tqdm(lst_0):
        for j in i:
            lst_1.append((j.split(' ')))
    
# 3. Computing tf -
    
    i = 0
    Values = []
    for sent in tqdm(range(len(lst_1))):
        Values.append([])
        for word in range(len(max_feature)):
            tf = lst_1[sent].count(max_feature[word])/len(lst_1[sent])

# 4. computing Idf-

            N = len(lst_1)     # total no. of documents in corpus

            n=0                # no. of doc has term-'t'
            for i in range(len(lst_1)):
                if lst_1[i].count(max_feature[word]) != 0:
                    n += 1

            idf = 1 + (np.log((1+N)/(1+n)))
            
# 5. Computing Tf-Idf values

            Tf_Idf = tf*idf
            Values[sent].append(Tf_Idf)
        
# 6. normalizing
    normalized_val = normalize(Values)
    
# 7. Creating sparse matrix
    sparse_val = csr_matrix(normalized_val)
    return sparse_val

In [24]:
b = transform(corpus, max_feature)

100%|████████████████████████████████████████████████████████████████████████████████| 746/746 [00:15<00:00, 47.55it/s]


In [39]:
b

<746x50 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Compressed Sparse Row format>

In [44]:
print('\nType of custom TfIdfVec output matrix =', type(b), '\n')

# tfidf matrix became very sparse due to only 50 features
print('printing some values of tfidf =\n',b[0], '\n')

print('Sparse matrix to Feature matrix =\n',b.toarray())


Type of custom TfIdfVec output matrix = <class 'scipy.sparse.csr.csr_matrix'> 

printing some values of tfidf =
   (0, 30)	1.0 

Sparse matrix to Feature matrix =
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Cross-check idf values using sklearn

In [45]:
vec1 = TfidfVectorizer().fit(corpus)
X1 = TfidfVectorizer().fit_transform(corpus)

features = vec1.get_feature_names()

sklearn_idf = vec1.idf_

In [49]:
(my_idf==sklearn_idf).all()

True

**Computing my_idf values**

In [48]:
# 1. Creating list of documents inside documents for simplification only
lst_0 = list()
j = 0
for i in corpus:
    lst_0.append([])
    lst_0[j].append(i)
    j += 1


# 2. Separating each word to iterate amongst words nicely
lst_1 = []
for i in lst_0:
    for j in i:
        lst_1.append((j.split(' ')))

# 3. Collecting unique words from corpus and sorting them

unique_words = set()
if isinstance(corpus, (list,)):
    for row in corpus:
        for word in row.split(' '):
            if len(word)<2:
                continue
            unique_words.add(word)
    unique_words = sorted(list(unique_words))

# 4. Looping row wise........

i = 0
idf_val = []
N = len(lst_1)             # total no. of documents in corpus
for sent in range(len(lst_1)):
    for word in range(len(unique_words)):
        
        n=0                # no. of doc has term-'t'
        for i in range(len(lst_1)):
            if lst_1[i].count(unique_words[word]) != 0:
                n += 1

        idf = 1 + (np.log((1+N)/(1+n)))
        idf_val.append(idf)
    break
my_idf = np.array(idf_val)